In [1]:
# Initialization
x0 = np.array([np.pi/2, np.pi/2, np.pi/4, -np.pi/2, -np.pi/4]) # Initial robot state

# Control loop definition
def control(x, param):
    f = fkin(x, param)
    J = Jkin(x, param)

    # u = np.linalg.pinv(J) @ (param.Mu - f) # Control commands
    # u = np.zeros(param.nbVarX) # Control commands

    pinvJ = np.linalg.inv(J.T @ J + np.eye(param.nbVarX) * 1E4) @ J.T # Damped pseudoinverse
    u = pinvJ @ (param.Mu - f) # Control commands

    return 0.1 * u / param.dt   # Velocity in rad/s

NameError: name 'np' is not defined

In [ ]:
# Initialization
x0 = np.array([np.pi/2, np.pi/2, np.pi/4, -np.pi/2, -np.pi/4]) # Initial robot state

# Control loop definition
def control(x, param):
    f = fkin(x, param)
    J = Jkin(x, param)

    # Prioritized control (left tracking as main objective)
    dfl = (param.Mu[:2] - f[:2]) # Left hand correction
    dfr = (param.Mu[2:] - f[2:]) # Right hand correction
    Jl = J[:2,:] # Jacobian for left hand
    Jr = J[2:,:] # Jacobian for right hand
    pinvJl = np.linalg.inv(Jl.T @ Jl + np.eye(param.nbVarX) * 1e1) @ Jl.T # Damped pseudoinverse
    Nl = np.eye(param.nbVarX) - pinvJl @ Jl # Nullspace projection operator
    ul = pinvJl @ dfl # Command for position tracking
    JrNl = Jr @ Nl
    pinvJrNl = JrNl.T @ np.linalg.inv(JrNl @ JrNl.T + np.eye(2) * 1e5) # Damped pseudoinverse
    ur = pinvJrNl @ (dfr - Jr @ ul) # Command for right hand tracking (with left hand tracking prioritized)
    u = ul + Nl @ ur # Control commands

    return 0.1 * u / param.dt   # Velocity in rad/s